In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob


In [2]:
#Load data

train = '/content/drive/MyDrive/566FinalProject/Problem_6_SLO_Fundus_Cup_and_Disc_Segmentation/FairSeg/Training'
test = '/content/drive/MyDrive/566FinalProject/Problem_6_SLO_Fundus_Cup_and_Disc_Segmentation/FairSeg/Testing'

trainfiles = glob.glob(train + '/*.npz')
testfiles = glob.glob(test + '/*.npz')

def load(files):
    fundus, mask, other = [], [], []
    for f in files:
        d = dict(np.load(f, allow_pickle=True))
        fundus_img = d['slo_fundus']
        mask_img = d['disc_cup_mask']
        mask_img = np.where(mask_img == -2, 2, np.where(mask_img == -1, 1, 0)).astype(np.uint8)
        fundus.append(fundus_img)
        mask.append(mask_img)
        other.append({
            'file': f,
            'age': float(d['age']),
            'gender': str(d['gender']),
            'race': str(d['race'])
        })
    return fundus, mask, other

train_fundus, train_mask, train_data = load(trainfiles)
test_fundus, test_mask, test_data = load(testfiles)

print("Training files:", len(train_fundus))
print("Testing files:", len(test_fundus))


Training files: 1000
Testing files: 500


In [3]:
#preprocess
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

preprocess_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

preprocess_test = ImageDataGenerator(rescale=1./255)

IMG_SIZE = 256

def resize_data(images, masks):
    x, y = [], []
    for img, mask in zip(images, masks):
        img = cv2.resize(img, (256, 256))
        mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)
        x.append(img)
        y.append(mask)
    return np.expand_dims(np.array(x), -1), np.array(y)

train_fundus, train_mask = resize_data(train_fundus, train_mask)
test_fundus, test_mask = resize_data(test_fundus, test_mask)

train_mask = tf.keras.utils.to_categorical(train_mask, num_classes=3)
test_mask = tf.keras.utils.to_categorical(test_mask, num_classes=3)

train_img = preprocess_train.flow(train_fundus, train_mask, batch_size=10, shuffle=True)
test_img = preprocess_test.flow(test_fundus, test_mask, batch_size=10, shuffle=False)


In [ ]:
print(train_fundus.shape)
print(test_fundus.shape)
print(train_mask.shape)
print(test_mask.shape)


(1000, 256, 256, 1)
(500, 256, 256, 1)
(1000, 256, 256, 3)
(500, 256, 256, 3)


In [4]:
#UNET
from tensorflow.keras import layers, models

def encoder(inputs):
  c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
  c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
  p1 = layers.MaxPooling2D((2, 2))(c1)

  c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
  c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
  p2 = layers.MaxPooling2D((2, 2))(c2)

  c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
  c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)
  p3 = layers.MaxPooling2D((2, 2))(c3)

  c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
  c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(c4)
  p4 = layers.MaxPooling2D((2, 2))(c4)

  return p4, [c1, c2, c3, c4]

def decoder(inputs, skips):
  c1, c2, c3, c4 = skips

  d1 = layers.UpSampling2D((2, 2))(inputs)
  d1 = layers.concatenate([d1, c4])
  c5 = layers.Conv2D(512, 3, activation='relu', padding='same')(d1)
  c5 = layers.Conv2D(512, 3, activation='relu', padding='same')(c5)

  d2 = layers.UpSampling2D((2, 2))(c5)
  d2 = layers.concatenate([d2, c3])
  c6 = layers.Conv2D(256, 3, activation='relu', padding='same')(d2)
  c6 = layers.Conv2D(256, 3, activation='relu', padding='same')(c6)

  d3 = layers.UpSampling2D((2, 2))(c6)
  d3 = layers.concatenate([d3, c2])
  c7 = layers.Conv2D(128, 3, activation='relu', padding='same')(d3)
  c7 = layers.Conv2D(128, 3, activation='relu', padding='same')(c7)

  d4 = layers.UpSampling2D((2, 2))(c7)
  d4 = layers.concatenate([d4, c1])
  c8 = layers.Conv2D(64, 3, activation='relu', padding='same')(d4)
  c8 = layers.Conv2D(64, 3, activation='relu', padding='same')(c8)

  return c8

def unet_model(input_shape=(256, 256, 1)):
  inputs = layers.Input(input_shape)
  b_input, skips = encoder(inputs)
  b = layers.Conv2D(1024, 3, activation='relu', padding='same')(b_input)
  b = layers.Conv2D(1024, 3, activation='relu', padding='same')(b)

  dec = decoder(b, skips)
  outputs = layers.Conv2D(3, 1, activation='softmax')(dec)

  return models.Model(inputs=[inputs], outputs=[outputs])


In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5,
    restore_best_weights = 'True'
)

model = unet_model(input_shape=(256,256,1))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy'],

)

history = model.fit(train_img, validation_data=test_img,  callbacks = [early_stopping], epochs=20)


Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step - accuracy: 0.9270 - loss: 0.4367 - val_accuracy: 0.9776 - val_loss: 0.0527
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.9772 - loss: 0.0614 - val_accuracy: 0.9776 - val_loss: 0.0452
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.9780 - loss: 0.0534 - val_accuracy: 0.9866 - val_loss: 0.0346
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.9817 - loss: 0.0453 - val_accuracy: 0.9902 - val_loss: 0.0303
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - accuracy: 0.9823 - loss: 0.0441 - val_accuracy: 0.9891 - val_loss: 0.0346
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.9834 - loss: 0.0414 - val_accuracy: 0.9908 - val_loss: 0.0282
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.9828 - loss: 0.0425 - val_accuracy: 0.9898 - val_loss: 0.0311
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.9832 - loss: 0.0415 - val_acc

In [10]:
import numpy as np

def dice_coef_per_class(y_true, y_pred, smooth=1e-6):
    dices = []
    for c in range(y_true.shape[-1]):
        y_true_f = y_true[..., c].flatten()
        y_pred_f = y_pred[..., c].flatten()
        intersection = np.sum(y_true_f * y_pred_f)
        dice = (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)
        dices.append(dice)
    return np.mean(dices)


preds = model.predict(test_fundus)
preds = np.argmax(preds, axis=-1)
test_true = np.argmax(test_mask, axis=-1)


overall_dice = dice_coef_per_class(tf.keras.utils.to_categorical(test_true, 3),
                                   tf.keras.utils.to_categorical(preds, 3))
print("Overall Dice Score:", overall_dice)


groups = {'Asian': [], 'Black': [], 'White': []}
race_map = {'0': 'Asian', '1': 'Black', '2': 'White'}

for i, info in enumerate(test_data):
    race_code = str(info['race']).strip()
    race = race_map.get(race_code, None)
    if race:
        true_mask = tf.keras.utils.to_categorical(test_true[i], 3)
        pred_mask = tf.keras.utils.to_categorical(preds[i], 3)
        groups[race].append(dice_coef_per_class(true_mask, pred_mask))

for g, vals in groups.items():
    if vals:
        print(f"{g} Dice Score: {np.mean(vals):.4f}")
    else:
        print(f"{g} Dice Score: No samples")


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step
Overall Dice Score: 0.8280418347207807
Asian Dice Score: 0.8124
Black Dice Score: 0.7958
White Dice Score: 0.8126




---



---



In [11]:
#FCN
import tensorflow as tf
from tensorflow.keras import layers, models

def fcn_model(input_shape=(256,256,1), num_classes=3):

    inputs = layers.Input(input_shape)


    x = layers.Conv2D(64, 3, padding="same", activation="relu")(inputs)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(256, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(2)(x)


    x = layers.Conv2D(512, 3, padding="same", activation="relu")(x)


    x = layers.Conv2DTranspose(256, 3, strides=2, padding="same", activation="relu")(x)
    x = layers.Conv2DTranspose(128, 3, strides=2, padding="same", activation="relu")(x)
    x = layers.Conv2DTranspose(64, 3, strides=2, padding="same", activation="relu")(x)


    outputs = layers.Conv2D(num_classes, 1, activation="softmax")(x)

    return models.Model(inputs, outputs)


In [12]:
import tensorflow as tf

train_fcn = tf.data.Dataset.from_tensor_slices((train_fundus, train_mask))
test_fcn  = tf.data.Dataset.from_tensor_slices((test_fundus, test_mask))

train_fcn = train_fcn.batch(10).prefetch(tf.data.AUTOTUNE)
test_fcn  = test_fcn.batch(10).prefetch(tf.data.AUTOTUNE)


In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5,
    restore_best_weights = 'True'
)

model_fcn = fcn_model(input_shape=(256,256,1), num_classes=3)

model_fcn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fcn = model_fcn.fit(
    train_fcn,
    validation_data=test_fcn,
    epochs=20,
    callbacks = [early_stopping]
)


Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.9025 - loss: 0.4192 - val_accuracy: 0.9778 - val_loss: 0.0709
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9789 - loss: 0.0633 - val_accuracy: 0.9831 - val_loss: 0.0485
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9847 - loss: 0.0455 - val_accuracy: 0.9876 - val_loss: 0.0364
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9876 - loss: 0.0363 - val_accuracy: 0.9892 - val_loss: 0.0316
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9888 - loss: 0.0329 - val_accuracy: 0.9899 - val_loss: 0.0289
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9896 - loss: 0.0302 - val_accuracy: 0.9904 - val_loss: 0.0274
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9901 - loss: 0.0282 - val_accuracy: 0.9908 - val_loss: 0.0263
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9906 - loss: 0.0265 - val_acc

In [14]:
import numpy as np
import tensorflow as tf

def dice_coef_per_class(y_true, y_pred, smooth=1e-6):
    dices = []
    for c in range(y_true.shape[-1]):
        y_true_f = y_true[..., c].flatten()
        y_pred_f = y_pred[..., c].flatten()
        intersection = np.sum(y_true_f * y_pred_f)
        dice = (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)
        dices.append(dice)
    return np.mean(dices)

preds_fcn = model_fcn.predict(test_fundus)
preds_fcn = np.argmax(preds_fcn, axis=-1)
test_true_fcn = np.argmax(test_mask, axis=-1)


overall_dice_fcn = dice_coef_per_class(
    tf.keras.utils.to_categorical(test_true_fcn, 3),
    tf.keras.utils.to_categorical(preds_fcn, 3)
)
print("FCN Overall Dice Score:", overall_dice_fcn)

groups_fcn = {'Asian': [], 'Black': [], 'White': []}
race_map = {'0': 'Asian', '1': 'Black', '2': 'White'}

for i, info in enumerate(test_data):
    race_code = str(info['race']).strip()
    race = race_map.get(race_code, None)
    if race:
        true_mask_i = tf.keras.utils.to_categorical(test_true_fcn[i], 3)
        pred_mask_i = tf.keras.utils.to_categorical(preds_fcn[i], 3)
        groups_fcn[race].append(dice_coef_per_class(true_mask_i, pred_mask_i))

for g, vals in groups_fcn.items():
    if vals:
        print(f"FCN {g} Dice Score: {np.mean(vals):.4f}")
    else:
        print(f"FCN {g} Dice Score: No samples")


16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 206ms/step
FCN Overall Dice Score: 0.8663714427632808
FCN Asian Dice Score: 0.8503
FCN Black Dice Score: 0.8178
FCN White Dice Score: 0.8626
